# Excercise 1 Invert the function

In [68]:
import math as m


def calc_discharge(b, h, m_bank, S_0, **kwargs):
    """
    Calculate the discahrge dependent on h
    """
    #auslesen des dictionary welches so aussieht {kwargs_name: kwargs_value}, umformung von den 3 möglichkeiten zu k_st
    for kwarg in kwargs.items():
        
        if "D_90" in kwarg[0]:
            k_st = 26 / (kwarg[1] ** (1/6))
            
        elif "k_st" in kwarg[0]:
            k_st = kwarg[1]
            
        elif "n_m" in kwarg[0]:
            k_st = 1 / kwarg[1]
            
    A = h * (b + h * m_bank)
    #berechnen von A mit m_bank und abhängig von h
    
    P = b + 2*h * m.sqrt(m_bank**2 + 1)
    #berechnen von P mit m_bank und abhängig von h
    
    Q = k_st * m.sqrt(S_0) * (A/P)**(2/3) * A
    #berechnen des Q abhängig von S_0 und h
    
    return Q,A,P,k_st
    #ausgabe von allen 4 wichtigen Werten
    
def interpolate_h(eps, Q, b, h, A, P, k_st, m_bank, S_0):
    """
    Invert the function, also add an emergency break for max iterations--> counter 1000
    """
    counter=0
    
    while eps > 10**-3:
        
        Qk = A ** (5/3) * m.sqrt(S_0) / (k_st * P ** (2 / 3))
        eps = abs(Q - Qk) / Q
        dA_dh = b + 2 * m_bank * h
        dP_dh = 2 * m.sqrt(m_bank ** 2 + 1)
        F = k_st * Q * P ** (2 / 3) - A ** (5 / 3) * m.sqrt(S_0)
        dF_dh = 2/3 * k_st * Q * P ** (-1 / 3) * dP_dh - 5 / 3 * A ** (2 / 3) * m.sqrt(S_0) * dA_dh
        h = abs(h - F / dF_dh)
        
        counter+=1
        #Emergency Stop for limit max iterations to 1000 counts
        if counter==1000:
            break
    
    return h,eps,Qk,counter
    
        
    

if __name__ == '__main__':
    # input parameters
    Q = 15.5        # discharge in (m3/s)
    b = 5.1         # bottom channel width (m)
    m_bank = 2.5    # bank slope
    #k_st = 20       # Strickler value
    #n_m = 1 / k_st  # Manning's n
    S_0 = 0.005     # channel slope
    eps = 1.0       # 

    h = float(input("Geben Sie eine Wasserhöhe an: "))         # water height
    
    kwargs_name = input("Geben Sie den Parameter: D_90, k_st oder n_m ein: ").strip()
    #kwargs_name ist string als Eingabe, .strip ist für Leerzeichen rausfiltern
    kwargs_value = float(input(f"Geben Sie den Wert für {kwargs_name} ein: "))
    #kwargs_value ist string, welcher konveritert wird und das f"" ermöglicht die eingabe von einem Value {} in einen string
    Q,A,P,k_st = calc_discharge(b, h, m_bank, S_0, **{kwargs_name: kwargs_value})
    #multiple return values mit , trennen und **{ : } so dictonary übergeben, sodass funktion das auslesen kann mit input
    print ("Discharge with h=%0.2fm is %0.2fm^3/s " %(h,Q)) 

    h,eps,Qk,counter = interpolate_h(eps, Q, b, h, A, P, k_st, m_bank, S_0)

    print ("The interpolate h is %0.2fm, with eps= %0.2f and Qk is %0.2fm^3/s and k_st= %0.2f with %0.2f iterations" %(h,eps,Qk,k_st,counter)) 
    # call the solver with user-defined channel geometry and discharge
    

Geben Sie eine Wasserhöhe an:  4
Geben Sie den Parameter: D_90, k_st oder n_m ein:  n_m
Geben Sie den Wert für n_m ein:  0.011


Discharge with h=4.00m Result:670.08m^3/s 
The interpolate h is 3.83m, with eps= 1.00 and Qk is 0.08 and k_st= 90.91 with 1000.00 iterations
